In [1]:
%pip install pandas folium

                                              0.0/102.3 kB ? eta -:--:--
     ----                                     10.2/102.3 kB ? eta -:--:--
     -----------                           30.7/102.3 kB 445.2 kB/s eta 0:00:01
     --------------                        41.0/102.3 kB 281.8 kB/s eta 0:00:01
     --------------                        41.0/102.3 kB 281.8 kB/s eta 0:00:01
     --------------                        41.0/102.3 kB 281.8 kB/s eta 0:00:01
     ----------------------                61.4/102.3 kB 218.8 kB/s eta 0:00:01
     ---------------------------------     92.2/102.3 kB 276.8 kB/s eta 0:00:01
     ------------------------------------ 102.3/102.3 kB 295.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import folium
import ast  # for string to tuple conversion

# Load the CSV
file_path = '..\data\exif_metadata.csv'  # replace with your file path
df = pd.read_csv(file_path)

# Convert string tuple representation to actual tuple
df['GPSLatitude'] = df['GPSLatitude'].apply(ast.literal_eval)
df['GPSLongitude'] = df['GPSLongitude'].apply(ast.literal_eval)

# Convert GPS coords to decimal format
def dms_to_decimal(degrees, minutes, seconds):
    return degrees + (minutes / 60) + (seconds / 3600)

df['DecimalLatitude'] = df.apply(lambda row: dms_to_decimal(*row['GPSLatitude']) * (-1 if row['GPSLatitudeRef'] == 'S' else 1), axis=1)
df['DecimalLongitude'] = df.apply(lambda row: dms_to_decimal(*row['GPSLongitude']) * (-1 if row['GPSLongitudeRef'] == 'W' else 1), axis=1)

# Plot the coords on a map
m = folium.Map(location=[0, 0], zoom_start=2)  # center the map at [0,0] and set a global zoom level

for _, row in df.iterrows():
    folium.CircleMarker(
        location=(row['DecimalLatitude'], row['DecimalLongitude']),
        radius=1,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.6,
    ).add_to(m)

m.save("dataset_ISS_location_map.html")
